In [1]:
!git clone https://github.com/microsoft/unilm.git
%cd unilm/speechlm

Cloning into 'unilm'...
remote: Enumerating objects: 9822, done.
remote: Counting objects: 100% (1780/1780), done.
remote: Compressing objects: 100% (431/431), done.
remote: Total 9822 (delta 1406), reused 1385 (delta 1349), pack-reused 8042
Receiving objects: 100% (9822/9822), 58.10 MiB | 37.61 MiB/s, done.
Resolving deltas: 100% (5063/5063), done.
/kaggle/working/unilm/speechlm


In [2]:
!pip install gdown
!gdown --fuzzy "https://drive.google.com/file/d/1azAQ_0AiwYSbmpwyTECCcraYKMkHBCxi/view?usp=sharing"

Downloading...
From (original): https://drive.google.com/uc?id=1azAQ_0AiwYSbmpwyTECCcraYKMkHBCxi
From (redirected): https://drive.google.com/uc?id=1azAQ_0AiwYSbmpwyTECCcraYKMkHBCxi&confirm=t&uuid=24a8a65c-fb5b-4fd4-8fb3-b995a1ab1ede
To: /kaggle/working/unilm/speechlm/speechlmh_base_checkpoint_clean.pt
100%|█████████████████████████████████████████| 389M/389M [00:03<00:00, 103MB/s]


In [3]:
import torch
import torch.nn.functional as F
from SpeechLM import SpeechLMConfig, SpeechLM


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# checkpoint = torch.load('speechlmh_base_checkpoint_clean.pt')
# cfg = SpeechLMConfig(checkpoint['cfg']['model'])
# model = SpeechLM(cfg)
# model.load_state_dict(checkpoint['model'])
# model = model.to(device)
# model.eval()

# wav_input_16khz = torch.randn(1,10000)
# normalize = checkpoint['cfg']['task']['normalize']  # False for base model, True for large model
# if normalize:
#     wav_input_16khz = F.layer_norm(wav_input_16khz[0], wav_input_16khz[0].shape).unsqueeze(0)

# # extract the representation of last layer
# rep = model.extract_features(wav_input_16khz.to(device))[0]

# # # extract the representation of each layer
# # output_layer = model.cfg.encoder_layers + model.cfg.text_transformer.encoder.layers
# # rep, layer_results = model.extract_features(wav_input_16khz, output_layer=output_layer, ret_layer_results=True)[0]
# # layer_reps = [x.transpose(0, 1) for x in layer_results]

In [4]:
import json


with open("/kaggle/input/tiny-stories-small/audio/train.json", "r") as f:
    train_info = json.load(f)

In [5]:
def get_features(wav1, wav2):
#     wav1 = F.pad(wav1, (0, max(0, wav2.shape[-1] - wav1.shape[-1])))
#     wav2 = F.pad(wav2, (0, max(0, wav1.shape[-1] - wav2.shape[-1])))
    
    if normalize:
        wav1 = F.layer_norm(wav1[0], wav1[0].shape).unsqueeze(0)
        wav2 = F.layer_norm(wav2[0], wav2[0].shape).unsqueeze(0)
        
    rep1 = model.extract_features(wav1.to(device))[0]
    rep2 = model.extract_features(wav2.to(device))[0]
    
    return rep1.mean(dim=1), rep2.mean(dim=1)

In [6]:
ind = 5

text = train_info[ind]["text"]
clean_path = train_info[ind]["clean path"]
aug_path = train_info[ind]["aug path"]

In [7]:
import torchaudio
import IPython.display as ipd

clean_wav, sr = torchaudio.load("/kaggle/input/tiny-stories-small/" + clean_path)
ipd.Audio(clean_wav, rate=sr)

In [8]:
aug_wav, sr = torchaudio.load("/kaggle/input/tiny-stories-small/" + aug_path)
ipd.Audio(aug_wav, rate=sr)

In [9]:
import torch.nn as nn

class SpeechLMMos(nn.Module):
    def __init__(self, freeze=True):
        super().__init__()
        checkpoint = torch.load('speechlmh_base_checkpoint_clean.pt')
        cfg = SpeechLMConfig(checkpoint['cfg']['model'])
        self.encoder = SpeechLM(cfg)
        self.encoder.load_state_dict(checkpoint['model'])
        self.encoder.eval()
        self.freeze = freeze
        
        self.dense = nn.Sequential(
            nn.Linear(768, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 1)
        )
        
        if self.freeze:
            self.encoder.eval()
            for p in self.encoder.parameters():
                p.requires_grad_(False)
        
    def forward(self, x):
        x = self.encoder.extract_features(x)[0] # [Batch, time, feats]
        x = self.dense(x) # [batch, time, 1]
        x = x.mean(dim=[1,2], keepdims=True) # [batch, 1, 1]
        return x
                

In [10]:
encoder = SpeechLMMos().to(device)

clean_score = encoder(clean_wav.to(device))
aug_score = encoder(aug_wav.to(device))

clean_score, aug_score

/opt/conda/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


(tensor([[[-0.0674]]], device='cuda:0', grad_fn=<MeanBackward1>),
 tensor([[[-0.0621]]], device='cuda:0', grad_fn=<MeanBackward1>))

In [11]:
def calc_rank_loss(clean_score, aug_score):
    p = torch.exp(clean_score - aug_score) / (1 + torch.exp(clean_score - aug_score))
    L_rank = - torch.log(p)
    return L_rank

In [12]:
loss = calc_rank_loss(clean_score, aug_score)
loss

tensor([[[0.6958]]], device='cuda:0', grad_fn=<NegBackward0>)